In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from  pmdarima import auto_arima
from matplotlib import pyplot as plt
from dateutil import relativedelta
import pickle

In [20]:
rub = {'Usd_to_Rub': [51.9749,50.0774,54.5285,56.9774,64.9363,67.9571,63.1248,66.6343,70.2244,76.523,79.4951,70.1542,66.4954,64.9306,66.0306,63.5773,64.3364,64.9737,62.9934,65.8591,60.8079,59.37,57.7388,58.954,56.2945,57.164,57.0303,59.8806,59.799,57.7706,57.6196,59.6207,58.7082,56.6019,57.5899,56.9372,61.4311,61.7684,62.2511,62.2934,66.7535,
68.2826,65.9751,67.9975,66.4337,67.192]}#vals of currency for 46 datapoints
fx = pd.DataFrame.from_dict(rub) # making it pandas
fx = fx.values.reshape(1,-1)
fx

array([[51.9749, 50.0774, 54.5285, 56.9774, 64.9363, 67.9571, 63.1248,
        66.6343, 70.2244, 76.523 , 79.4951, 70.1542, 66.4954, 64.9306,
        66.0306, 63.5773, 64.3364, 64.9737, 62.9934, 65.8591, 60.8079,
        59.37  , 57.7388, 58.954 , 56.2945, 57.164 , 57.0303, 59.8806,
        59.799 , 57.7706, 57.6196, 59.6207, 58.7082, 56.6019, 57.5899,
        56.9372, 61.4311, 61.7684, 62.2511, 62.2934, 66.7535, 68.2826,
        65.9751, 67.9975, 66.4337, 67.192 ]])

In [29]:
data = pd.read_csv(r'C:\Users\work2\Desktop\Raisk\Регламент\Статья про mixed models\aver card spending.csv', delimiter = ';')
#extract currency as feat
fx2 = data.iloc[-1]
type(fx2)
fx2

Reg           usd to rub
15.01.2013       30,2607
15.02.2013       30,0773
15.03.2013       30,7769
15.04.2013       30,9308
                 ...    
15.09.2018       68,2826
15.10.2018       65,9751
15.11.2018       67,9975
15.12.2018       66,4337
15.01.2019        67,192
Name: 83, Length: 74, dtype: object

In [17]:
x = np.repeat(np.array([[74.7]]),24, axis=1)
x

array([[74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7,
        74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7, 74.7,
        74.7, 74.7]])

In [55]:
data2 = pd.read_csv(r'C:\Users\work2\Desktop\Raisk\Регламент\Статья про mixed models\opendata.csv', quotechar='"', encoding='cp1251', delimiter = ',')

   
rest_exp = data2[(data2['name'] == 'Средние траты в ресторане')]
rest_exp['date'] = pd.to_datetime(data2['date']) #convert to date
for reg in rest_exp.region.unique(): #unique regions in row
    rest_exp_for_arimax = rest_exp[(rest_exp.region == reg)]['value'] #filter on region, getting only val column
    rest_exp_for_arimax = np.log(rest_exp_for_arimax)#log it
    rest_exp_for_arimax = rest_exp_for_arimax.diff() #first-order-difference
    rest_exp_for_arimax = rest_exp_for_arimax.dropna(0) #drop NaNs for auto-arima
    arima = auto_arima(rest_exp_for_arimax,exogenous=fx.T, error_action='ignore', seasonal=True, m=12) #auto arima tool with seasonal vals
    print(str(arima.summary()),  file=open('Средние траты в ресторане' + ' '+  reg + '.txt', 'w')) #saving params in txt
    with open('Средние траты в ресторане' +' ' + reg + ' arimaX.pkl', 'wb') as pkl: #dump the model
        pickle.dump(arima, pkl)
    #plt.scatter(x, rest_exp_for_arimax, marker='x') #plot model and x at the same time
    #plt.plot(x, arima.predict(n_periods=rest_exp_for_arimax.shape[0]))
    #plt.title(reg)
    #plt.show()
    new_y = arima.predict(n_periods=24,exogenous =np.repeat(np.array([[85]]),24, axis=1).T) #24-months forward with repeating exog var - new currency rate
    sorted_by_date = rest_exp[(rest_exp.region == reg)][['date','value']] #tmp save
    sorted_by_date.sort_values('date', inplace=True, ascending=False) # now sort
    temp_next_val = sorted_by_date[:1]['value'] + sum(np.exp(np.cumsum(new_y)))#summing anti-diff from arima model
    temp_next_date  = sorted_by_date[:1]['date'].dt.date + relativedelta.relativedelta(months =+24)#adding 24 months (auto-arima pred) to last date
    #temp_row = ['Средние траты в ресторане',reg,temp_next_date.values[0],temp_next_val.values[0]] #temporary row to rest_exp
    temp_pandas = pd.DataFrame([['Средние траты в ресторане',reg,temp_next_date.values[0],temp_next_val.values[0]]], columns =['name', 'region','date','value'])
    rest_exp= pd.concat([rest_exp,temp_pandas])
rest_exp.to_excel('Средние траты в ресторане.xlsx')

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\sit

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estima

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving avera

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estima

...

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Usin

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Using zeros as

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1006: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starti

C:\Users\work2\AppData\Local\conda\conda\envs\Anaconda3_64\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [62]:
#теперь часть mixedLM но она вне цикла
model = sm.MixedLM(rest_exp,exog=fx.astype(float), groups=rest_exp.region)
result = model.fit()
print(result.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [57]:
rest_exp

,name,region,date,value
70894,Средние траты в ресторане,Россия,2015-01-15 00:00:00,1901.000000
70895,Средние траты в ресторане,Россия,2015-02-15 00:00:00,1836.000000
70896,Средние траты в ресторане,Россия,2015-03-15 00:00:00,1864.000000
70897,Средние траты в ресторане,Россия,2015-04-15 00:00:00,1798.000000
70898,Средние траты в ресторане,Россия,2015-05-15 00:00:00,1830.000000
...,...,...,...,...
0,Средние траты в ресторане,Чеченская Республика,2020-11-15,2114.672201
0,Средние траты в ресторане,Чувашская Республика,2020-11-15,1112.178575
0,Средние траты в ресторане,Чукотский автономный округ,2020-11-15,2350.089920
0,Средние траты в ресторане,Ямало-Ненецкий автономный округ,2020-11-15,1809.154777
